[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0087DeepLearningConv2DFashionMNIST.ipynb)

The data can be downloaded from:
https://www.kaggle.com/datasets/dschettler8845/brats-2021-task1/data

In [1]:
# Import Packages
# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# Deep Learning
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import torchinfo
from torchmetrics.classification import MulticlassAccuracy
import torchvision

# Miscellaneous
import math
import os
from dotenv import load_dotenv

from platform import python_version
from platform import python_version
import random
import time

# Typing
from typing import Callable, Dict, Generator, List, Optional, Self, Set, Tuple, Union

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import HTML, Image
from IPython.display import display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout, SelectionSlider
from ipywidgets import interact

import pytest
import nibabel as nib
import os
from nibabel.testing import data_path
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from skimage.util import montage 
from skimage.transform import rotate
from sklearn import preprocessing as pre
from keras.utils import to_categorical 



## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [2]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

# Improve performance by benchmarking
torch.backends.cudnn.benchmark = True

# Reproducibility
# torch.manual_seed(seedNum)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark     = False


In [3]:
# Constants


IMG_SIZE = (240,240,155)

TENSOR_BOARD_BASE   = 'TB'



In [4]:
# Download Auxiliary Modules for Google Colab
if runInGoogleColab:
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DataManipulation.py
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DataVisualization.py
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DeepLearningPyTorch.py

In [5]:
# Courses Packages

from DataVisualization import PlotLabelsHistogram, PlotMnistImages
from DeepLearningPyTorch import TrainModel


In [6]:
if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
 dev = "cpu" 
 device = torch.device(dev)


In [7]:
load_dotenv()

def str_to_bool(s):
    return s.lower() in ('true', '1')
print(os.getenv("ROOT_DIR"))

E:/BRaTS_2021/BRaTS_2021_task_1/


## Brain Tumor Image Classification using DL




In [8]:
# Parameters

# Data
numSamplesTrain = 0 #TBD after we know how many samples we have
numSamplesTest  = 0 #TBD after we know how many samples we have
trainTestPercentage = 0.2
# Model
dropP = 0.2 #<! Dropout Layer

# Training
batchSize   = 10
numWork     = 2 #<! Number of workers
nEpochs     = 30

# Visualization
numImg = 3

# samples root directory
root_dir = os.getenv("ROOT_DIR") #The directeroy of the original Kaggel archive. 
numpy_archive_path = os.getenv("NUMPY_ARCHIVE_PATH") #the root directory of the processed numpy samples 
image_path = numpy_archive_path +'new_archive_cat/images/'  # path to the processed numpy images 
label_path = numpy_archive_path +'new_archive_cat/labels/'  # path to the processed numpy labeles 


# image types
img_types = ['t1','t1ce','t2','flair','seg']


#number of pixels in an image
p_image_size = IMG_SIZE[0]*IMG_SIZE[1]*IMG_SIZE[2]
crop_size = (130,160,128)


## Generate Data

* Read the Kaggle archive (https://www.kaggle.com/datasets/dschettler8845/brats-2021-task1) and process it to numpy images to be saved to the disk.
* Those functions are used only once to generate the data.
* If you have the processed data already skip this section.

In [9]:
#These functions to be run once to generate the numpy files containing the images
def get_file_lists():
   files = os.listdir(root_dir)
   
   file_list = []
   
   for file_name in files:
     if os.path.isdir(root_dir +file_name):
      all_samples_dirs = os.listdir(root_dir + file_name +'/')
      for nii_dir in all_samples_dirs:
        if os.path.isdir(root_dir + file_name + '/'+nii_dir):
           nii_files = os.listdir(root_dir +'/'+ file_name + '/'+nii_dir)
           
           local_dir = root_dir + file_name + '/'+nii_dir+'/'
           sample_files=['','','','','']
           for file in nii_files:
            if '_t1.' in file: 
               sample_files[0] = local_dir+file
            elif '_t1ce.' in file: 
               sample_files[1] = local_dir+file
            elif '_t2.' in file: 
               sample_files[2] = local_dir+file
            elif '_flair.' in file: 
               sample_files[3] = local_dir+file
            elif '_seg.' in file: 
               sample_files[4] = local_dir+file
            else:
               print (file)
           file_list.append(sample_files)

   return np.array(file_list)

def create_repository():
    # Directory 
    archive_directory = "new_archive_cat"
    
    # Path 
    root_archive_path = os.path.join(numpy_archive_path, archive_directory) 
    os.mkdir(root_archive_path) 
    
    # Directory 
    image_directory = "images"
   
    # Path 
    image_path = os.path.join(root_archive_path, image_directory) 
    os.mkdir(image_path) 
    
    # Directory 
    label_directory = "labels"
   
    # Path 
    label_path = os.path.join(root_archive_path, label_directory) 
    os.mkdir(label_path) 

#test function
def test_combined():
#1. Normalize the images between 0 to 1
      #t1
      sample_files = [f'{root_dir}\BraTS2021_Training_Data\BraTS2021_01666\BraTS2021_01666_t1.nii.gz',
                      f'{root_dir}\BraTS2021_Training_Data\BraTS2021_01666\BraTS2021_01666_t1ce.nii.gz',
                      f'{root_dir}\BraTS2021_Training_Data\BraTS2021_01666\BraTS2021_01666_t2.nii.gz',
                      f'{root_dir}\BraTS2021_Training_Data\BraTS2021_01666\BraTS2021_01666_flair.nii.gz',
                      f'{root_dir}\BraTS2021_Training_Data\BraTS2021_01666\BraTS2021_01666_seg.nii.gz']
      t1 = nib.load(sample_files[0]).get_fdata()
      t1_norm = t1.reshape(-1,t1.shape[-1])
      t1_norm = pre.MinMaxScaler().fit_transform(t1_norm)
      t1_norm = t1_norm.reshape(*IMG_SIZE)
      
      t1ce = nib.load(sample_files[1]).get_fdata()
      t1ce_norm = t1ce.reshape(-1,t1ce.shape[-1])
      t1ce_norm = pre.MinMaxScaler().fit_transform(t1ce_norm)
      t1ce_norm = t1ce_norm.reshape(*IMG_SIZE)
      
      t2 = nib.load(sample_files[2]).get_fdata()
      t2_norm = t2.reshape(-1,t2.shape[-1])
      t2_norm = pre.MinMaxScaler().fit_transform(t2_norm)
      t2_norm = t2_norm.reshape(*IMG_SIZE)
      
      flair = nib.load(sample_files[3]).get_fdata()
      flair_norm = flair.reshape(-1,flair.shape[-1])
      flair_norm = pre.MinMaxScaler().fit_transform(flair_norm)
      flair_norm = flair_norm.reshape(*IMG_SIZE)
      
      #2. Get the mask file and change its labels to 0-3 
      seg = nib.load(sample_files[4]).get_fdata()
      seg = seg.astype(np.uint8)
      seg[seg==4] = 3
      
      # 3. combine the images into one with 4 channels
      combined_image = np.stack([t1_norm,t1ce_norm,t2_norm,flair_norm],axis=3)
      
      # 4. Crop the images to 125x125x125x4
      combined_image= combined_image[IMG_SIZE[0]//2 - crop_size//2:IMG_SIZE[0]//2 +crop_size//2,
                                     IMG_SIZE[1]//2 - crop_size//2:IMG_SIZE[1]//2 + crop_size//2,
                                     IMG_SIZE[2]//2 - crop_size//2:IMG_SIZE[2]//2 + crop_size//2]
      
      #5. crop the mask
      seg = seg[IMG_SIZE[0]//2 - crop_size//2:IMG_SIZE[0]//2 +crop_size//2,
                IMG_SIZE[1]//2 - crop_size//2:IMG_SIZE[1]//2 + crop_size//2,
                IMG_SIZE[2]//2 - crop_size//2:IMG_SIZE[2]//2 + crop_size//2]

      return combined_image
   
def calc_mean_std():
   file_list = os.listdir(image_path)
   avg_list= np.zeros(len(file_list)*4)
   avg_list = avg_list.reshape(len(file_list),4)
   std_list= np.zeros(len(file_list)*4)
   std_list = std_list.reshape(len(file_list),4)
   i=0
   for sample in file_list:
    image = np.load(image_path+sample)
    avg_list[i] = np.mean(image,axis=(0,1,2))
    std_list[i] = np.std(image,axis=(0,1,2))
    i+=1
   
   return np.mean((avg_list),axis=0),np.mean((std_list),axis=0)

In [10]:

# Set the environment variables for enabling/disabling each channel
ENABLE_T1 = str_to_bool(os.getenv('ENABLE_T1'))
ENABLE_T1CE = str_to_bool(os.getenv('ENABLE_T1CE'))
ENABLE_T2 = str_to_bool(os.getenv('ENABLE_T2'))
ENABLE_FLAIR = str_to_bool(os.getenv('ENABLE_FLAIR'))
print(f"ENABLE_T1: {ENABLE_T1} | ENABLE_T1CE: {ENABLE_T1CE} | ENABLE_T2: {ENABLE_T2} | ENABLE_FLAIR: {ENABLE_FLAIR}")

IMG_SIZE = (240, 240, 155)  # Example size, replace with actual size
image_path = './images/'  # Replace with actual path
label_path = './labels/'  # Replace with actual path

def generate_images(file_list):
    sample_number = 0
    start_time = time.time()
    for sample_files in file_list:
        combined_images_list = []

        if ENABLE_T1:
            t1 = nib.load(sample_files[0]).get_fdata()
            t1_norm = t1.reshape(-1, t1.shape[-1])
            t1_norm = pre.MinMaxScaler().fit_transform(t1_norm)
            t1_norm = t1_norm.reshape(*IMG_SIZE)
            combined_images_list.append(t1_norm)

        if ENABLE_T1CE:
            t1ce = nib.load(sample_files[1]).get_fdata()
            t1ce_norm = t1ce.reshape(-1, t1ce.shape[-1])
            t1ce_norm = pre.MinMaxScaler().fit_transform(t1ce_norm)
            t1ce_norm = t1ce_norm.reshape(*IMG_SIZE)
            combined_images_list.append(t1ce_norm)

        if ENABLE_T2:
            t2 = nib.load(sample_files[2]).get_fdata()
            t2_norm = t2.reshape(-1, t2.shape[-1])
            t2_norm = pre.MinMaxScaler().fit_transform(t2_norm)
            t2_norm = t2_norm.reshape(*IMG_SIZE)
            combined_images_list.append(t2_norm)

        if ENABLE_FLAIR:
            flair = nib.load(sample_files[3]).get_fdata()
            flair_norm = flair.reshape(-1, flair.shape[-1])
            flair_norm = pre.MinMaxScaler().fit_transform(flair_norm)
            flair_norm = flair_norm.reshape(*IMG_SIZE)
            combined_images_list.append(flair_norm)

        # Get the mask file and change its labels to 0-3 
        seg = nib.load(sample_files[4]).get_fdata()
        seg = seg.astype(np.uint8)
        seg[seg == 4] = 3

        # Combine the images into one with multiple channels
        combined_image = np.stack(combined_images_list, axis=3)

        # Crop the images to 130x160x125xN
        combined_image = combined_image[55:185, 43:203, 13:141]

        # Crop the mask
        seg = seg[55:185, 43:203, 13:141]
        seg = to_categorical(seg, 4)

        # Save the files
        np.save(image_path + 'sample_' + str(sample_number) + '.npy', combined_image)
        np.save(label_path + 'label_' + str(sample_number) + '.npy', seg)
        print(f'Sample number {sample_number + 1} saved. Total samples {len(file_list)}')
        sample_number += 1
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'Elapsed time: {elapsed_time/60} minutes')

ENABLE_T1: False | ENABLE_T1CE: True | ENABLE_T2: True | ENABLE_FLAIR: True


Run this cell if you want to create the numpy image repository

In [12]:
#create_repository and generate images
if str_to_bool(os.getenv('CREATE_REPOSITORY')):
    create_repository()
    files = get_file_lists()
    generate_images(files)

FileNotFoundError: [Errno 2] No such file or directory: './images/sample_0.npy'

In [ ]:
if str_to_bool(os.getenv('TEST_COMBINED')):
    test_combined()

get the list of files

In [ ]:
mX = os.listdir(image_path)
vY = os.listdir(label_path)
mX.sort()
vY.sort()


In [ ]:
print(mX[555],vY[555])

In [ ]:
numSamplesTest = np.int32(np.round(len(mX) * trainTestPercentage))
numSamplesTrain = len(mX) - numSamplesTest

print("Number of Train samples:",numSamplesTrain)
print("Number of Test samples:",numSamplesTest)

## Define the dataset and Loader

The dataset takes as parameters:
* mX - path to the sample numpy image file
* vY - path to the masked label file


In [ ]:


###################################################################
#
# The format of the batch returned:
# Image: batch_size x number of images (4) x image_size (240x240x155)
# Label: batch_size x number of images (1) x image_size (240x240x155)
###################################################################

class ImageDatasetFromDisk(Dataset):
    def __init__(self, mX, vY, transform=None, target_transform=None):
        
        self.images = mX
        self.labels = vY
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        
        image = np.load(image_path+self.images[idx])
        
        label = np.load(label_path+self.labels[idx])
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

## Pre Process Data

This section normalizes the data to have zero mean and unit variance per **channel**.  
It is required to calculate:

 * The average pixel value per channel.
 * The standard deviation per channel.

In [ ]:
# Calculate the Standardization Parameters

#===========================Fill This===========================#
# 1. Calculate the mean per channel.
# 2. Calculate the standard deviation per channel.

#file_list = get_file_names_by_type(root_dir,'HGG')
#µ  = [np.mean(mX[:,:p_image_size]),np.mean(mX[:,p_image_size:p_image_size*2]),np.mean(mX[:,p_image_size*2:p_image_size*3]),np.mean(mX[:,p_image_size*3:p_image_size*4])]
#σ  = [np.std(mX[:,:p_image_size]),np.std(mX[:,p_image_size:p_image_size*2]),np.std(mX[:,p_image_size*2:p_image_size*3]),np.std(mX[:,p_image_size*3:p_image_size*4])]

#===============================================================#


µ = [0.32519394, 0.17367085, 0.18451846, 0.23585342]
σ = [0.32613024, 0.17851571, 0.20511049, 0.25051804]


### Plot the Data

In [ ]:
# Histogram of Labels - TBD

#hA = PlotLabelsHistogram(vY, lClass = L_CLASSES_FASHION_MNIST)
#plt.show()

## Train & Test Split

In [ ]:
# Train Test Split - TBD

#numClass = len(np.unique(vY))

#===========================Fill This===========================#
# 1. Split the data into train and test (Validation) data sets (NumPy arrays).
# 2. Use stratified split.
# !! The output should be: `mXTrain`, `mXTest`, `vYTrain`, `vYTest`.

mXTrain, mXTest, vYTrain, vYTest = train_test_split(mX, vY, test_size = numSamplesTest, train_size = numSamplesTrain, shuffle = True)
#===============================================================#

print(f'The training features data shape: {len(mXTrain)}')
print(f'The training labels data shape: {len(vYTrain)}')
print(f'The test features data shape: {len(mXTest)}')
print(f'The test labels data shape: {len(vYTest)}')
#print(f'The unique values of the labels: {np.unique(vY)}')


Create the dataset

In [ ]:

mXTrain.sort()
vYTrain.sort()
mXTest.sort()
vYTest.sort()
dsTrain = ImageDatasetFromDisk(mXTrain,vYTrain)
dsTest = ImageDatasetFromDisk(mXTest,vYTest)

print(f'The training data set data len: {(len(dsTrain))}')
print(f'The test data set data len: {(len(dsTest))}')

In [ ]:
# create transform - convert image to tensor

class ToTensor(object):
    """Convert image in sample to Tensors."""

    def __call__(self, image):
         
        tImg = torch.from_numpy(image).permute(3,0,1,2)
        
        return tImg


In [ ]:
# Update Transformer

#===========================Fill This===========================#
# 1. Define a transformer which normalizes the data.
oDataTrns = torchvision.transforms.Compose([  #<! Chaining transformations
    ToTensor(),        #<! Convert to Tensor (4,130,160,125)
    torchvision.transforms.Normalize(µ[-1], σ[-1]),   #<! Normalizes the Data (https://pytorch.org/vision/main/generated/torchvision.transforms.Normalize.html)
    ])

oLblTrns = torchvision.transforms.Compose([  #<! Chaining transformations
    ToTensor(),        #<! Convert to Tensor (1,130,160,125)
    ])

# Update the DS transformer
dsTrain.transform = oDataTrns
dsTrain.target_transform  = oLblTrns

dsTest.transform = oDataTrns
dsTest.target_transform  = oLblTrns
#===============================================================#


### Data Loaders

The dataloader is the functionality which loads the data into memory in batches.  
Its challenge is to bring data fast enough so the Hard Disk is not the training bottleneck.  
In order to achieve that, Multi Threading / Multi Process is used.


In [ ]:
# Data Loader

#===========================Fill This===========================#
# 1. Create the train data loader.
# 2. Create the test data loader.
# !! Think about the values of `shuffle` and `batch_size` for the train / test.
dlTrain  = torch.utils.data.DataLoader(dsTrain, shuffle = True, batch_size = 1 * batchSize,drop_last=True)
dlTest   = torch.utils.data.DataLoader(dsTest, shuffle = False, batch_size = 2 * batchSize,drop_last=True)

#===============================================================#


In [ ]:
# Iterate on the Loader
# The first batch.
tX, vY = next(iter(dlTrain)) #<! PyTorch Tensors

print(f'The batch features dimensions: {tX.shape}')
print(f'The batch labels dimensions: {vY.shape}')

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (12, 6))
x = vY[0]
x = np.argmax(x,axis=0)
ax1.imshow(x[:,:,75])
ax2.imshow(tX[0,3,:,:,75])

## Define the Model

This section build 3 different models to be evaluated.

In [ ]:
# Model Container
lModels = []

In [ ]:
# Model
# Defining a sequential model.

oModel = nn.Sequential(
#===========================Fill This===========================#
# 1. Create the 1st model.
# 2. Use 3 layers.
# !! You may use different kernel size, dropout probability, max pooling, etc...

    nn.Identity(),
    
    nn.Conv2d(in_channels = 1, out_channels = 30, kernel_size = 3, bias = False),
    nn.BatchNorm2d(num_features = 30),
    nn.ReLU(),
    nn.Dropout2d(p = dropP),
    
    nn.Conv2d(in_channels = 30, out_channels = 60, kernel_size = 3, bias = False),
    nn.MaxPool2d(kernel_size = 2),
    nn.BatchNorm2d(num_features = 60),
    nn.ReLU(),
    nn.Dropout2d(p = dropP),
            
    nn.Conv2d(in_channels = 60,  out_channels = 120, kernel_size = 3, bias = False),
    nn.MaxPool2d(kernel_size = 2),
    nn.BatchNorm2d(num_features = 120),
    nn.ReLU(),
    nn.Dropout2d(p = dropP),
    
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(120, len(L_CLASSES_FASHION_MNIST)),
#===============================================================#
)

print(torchinfo.summary(oModel, tX.shape, col_names = ['kernel_size', 'output_size', 'num_params'], device = 'cpu')) #<! Added `kernel_size`

# Append Model
lModels.append(oModel)

In [ ]:
# Model
# Defining a sequential model.

oModel = nn.Sequential(
#===========================Fill This===========================#
# 1. Create the 2nd model.
# 2. Use 3 layers.
# !! You may use different kernel size, dropout probability, max pooling, etc...

    nn.Identity(),
    
    nn.Conv2d(in_channels = 1, out_channels = 30, kernel_size = 5, bias = False),
    nn.BatchNorm2d(num_features = 30),
    nn.ReLU(),
    nn.Dropout2d(p = dropP),
    
    nn.Conv2d(in_channels = 30, out_channels = 60, kernel_size = 5, bias = False),
    nn.BatchNorm2d(num_features = 60),
    nn.ReLU(),
    nn.Dropout2d(p = dropP),
            
    nn.Conv2d(in_channels = 60,  out_channels = 120, kernel_size = 5, bias = False),
    nn.BatchNorm2d(num_features = 120),
    nn.ReLU(),
    nn.Dropout2d(p = dropP),
    
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(120, len(L_CLASSES_FASHION_MNIST)),
#===============================================================#
)

print(torchinfo.summary(oModel, tX.shape, col_names = ['kernel_size', 'output_size', 'num_params'], device = 'cpu')) #<! Added `kernel_size`

# Append Model
lModels.append(oModel)

In [ ]:
# Model
# Defining a sequential model.

oModel = nn.Sequential(
#===========================Fill This===========================#
# 1. Create the 3rd model.
# 2. Use 3 layers.
# !! You may use different kernel size, dropout probability, max pooling, etc...

    nn.Identity(),
    
    nn.Conv2d(in_channels = 1, out_channels = 30, kernel_size = 7, bias = False),
    nn.MaxPool2d(kernel_size = 2),
    nn.BatchNorm2d(num_features = 30),
    nn.ReLU(),
    nn.Dropout2d(p = dropP),
    
    nn.Conv2d(in_channels = 30, out_channels = 60, kernel_size = 5, bias = False),
    nn.MaxPool2d(kernel_size = 2),
    nn.BatchNorm2d(num_features = 60),
    nn.ReLU(),
    nn.Dropout2d(p = dropP),
            
    nn.Conv2d(in_channels = 60,  out_channels = 120, kernel_size = 3, bias = False),
    nn.BatchNorm2d(num_features = 120),
    nn.ReLU(),
    nn.Dropout2d(p = dropP),
    
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(120, len(L_CLASSES_FASHION_MNIST)),
#===============================================================#
)

print(torchinfo.summary(oModel, tX.shape, col_names = ['kernel_size', 'output_size', 'num_params'], device = 'cpu')) #<! Added `kernel_size`

# Append Model
lModels.append(oModel)

* <font color='brown'>(**#**)</font> Guideline: The smaller the image gets, the deeper it is (More channels).   
  The intuition, the beginning of the model learns low level features (Small number), deeper learns combinations of features (Larger number).

## Training Loop


### Train the Model

In [ ]:
# Check GPU Availability

runDevice   = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') #<! The 1st CUDA device


In [ ]:
# Set the Loss & Score

#===========================Fill This===========================#
# 1. Define loss function
# 2. Define score function.
hL = nn.CrossEntropyLoss()
hS = MulticlassAccuracy(num_classes = len(L_CLASSES_FASHION_MNIST))
hL = hL.to(runDevice) #<! Not required!
hS = hS.to(runDevice)
#===============================================================#

In [ ]:
# Train the Model

#===========================Fill This===========================#
# 1. Build a loop to evaluate all models.
# 2. Define a TensorBoard Writer per model to keep its score.
# !! You may use `TrainModel()`.

for ii, oModel in enumerate(lModels):
    # Hyper Parameter Loop
    oTBWriter = SummaryWriter(log_dir = os.path.join(TENSOR_BOARD_BASE, f'Model{(ii + 1):03d}'))
    oModel = oModel.to(runDevice) #<! Transfer model to device
    oOpt = torch.optim.AdamW(oModel.parameters(), lr = 6e-4, betas = (0.9, 0.99), weight_decay = 1e-3) #<! Define optimizer
    oRunModel, lTrainLoss, lTrainScore, lValLoss, lValScore = TrainModel(oModel, dlTrain, dlTest, oOpt, nEpochs, hL, hS, oTBWriter)
    oTBWriter.close()
#===============================================================#

* <font color='brown'>(**#**)</font> Run `tensorboard --logdir=TB` from the Jupyter notebook path.
* <font color='green'>(**@**)</font> Optimize the model search to get above 92% accuracy in validation set.